In [2]:
import ee
import os
import time

# ✅ Initialize Earth Engine
ee.Initialize(project='ee-felixlundfrandsen')

# ✅ Define the region of interest (Example: California)
roi = ee.Geometry.Rectangle([-120.0, 37.0, -119.5, 37.5])  # Example: California

# ✅ Change the Year Here
year = 2022  # Select the desired year

# ✅ Load NAIP High-Resolution Aerial Imagery (0.6m per pixel) for the Selected Year
naip = ee.ImageCollection("USDA/NAIP/DOQQ") \
    .filterDate(f"{year}-01-01", f"{year}-12-31")  # ✅ Select specific year
naip = naip.filterBounds(roi).median().select(["R", "G", "B"])  # ✅ Red, Green, Blue bands

# ✅ Set Scale for NAIP (0.6m per pixel)
scale = 0.6  # Best available free resolution

# ✅ Define Tile Size (256 pixels at 0.6m per pixel)
grid_size = 256 * scale  # Tile covers 153.6m × 153.6m

# Function to generate bounding boxes for tiles
def create_tiles(roi, grid_size_meters):
    tiles = []
    coords = roi.bounds().coordinates().getInfo()[0]
    lon_min, lat_min = coords[0]  # Lower-left corner
    lon_max, lat_max = coords[2]  # Upper-right corner

    num_tiles_x = int((lon_max - lon_min) / (grid_size_meters / 111320))  # Convert meters to degrees
    num_tiles_y = int((lat_max - lat_min) / (grid_size_meters / 111320))

    for i in range(num_tiles_x):
        for j in range(num_tiles_y):
            lon1 = lon_min + i * (grid_size_meters / 111320)
            lat1 = lat_min + j * (grid_size_meters / 111320)
            lon2 = min(lon1 + (grid_size_meters / 111320), lon_max)
            lat2 = min(lat1 + (grid_size_meters / 111320), lat_max)
            bbox = ee.Geometry.Rectangle([lon1, lat1, lon2, lat2])
            tiles.append(bbox)

    return tiles

# ✅ Create tiles (based on selected zoom level)
tiles = create_tiles(roi, grid_size)

# ✅ Limit to 10 images
tiles = tiles[:10]

# ✅ Export images to Google Drive
folder_name = f"EarthEngine_NAIP_{year}"  # Save images in a folder named by year
task_list = []

for i, bbox in enumerate(tiles):
    file_name = f"{year}_tile_{i}.tif"  # Save as GeoTIFF

    task = ee.batch.Export.image.toDrive(
        image=naip,
        description=file_name,
        folder=folder_name,
        fileNamePrefix=file_name.replace(".tif", ""),
        region=bbox,
        crs="EPSG:4326",
        scale=scale,  # ✅ 0.6m per pixel resolution
        maxPixels=1e13  # Avoid pixel limits
    )
    
    task.start()
    task_list.append(task)
    print(f"🚀 Export started: {file_name} (Check Google Drive)")

# ✅ Monitor export tasks
print("\n📡 Waiting for tasks to complete...")
while any(task.status()['state'] in ['READY', 'RUNNING'] for task in task_list):
    time.sleep(30)  # Wait before checking again
    statuses = [task.status()['state'] for task in task_list]
    print(f"🔄 Current Task Status: {statuses}")

print(f"\n✅ All exports for {year} completed! Check Google Drive in '{folder_name}' folder.")


🚀 Export started: 2022_tile_0.tif (Check Google Drive)
🚀 Export started: 2022_tile_1.tif (Check Google Drive)
🚀 Export started: 2022_tile_2.tif (Check Google Drive)
🚀 Export started: 2022_tile_3.tif (Check Google Drive)
🚀 Export started: 2022_tile_4.tif (Check Google Drive)
🚀 Export started: 2022_tile_5.tif (Check Google Drive)
🚀 Export started: 2022_tile_6.tif (Check Google Drive)
🚀 Export started: 2022_tile_7.tif (Check Google Drive)
🚀 Export started: 2022_tile_8.tif (Check Google Drive)
🚀 Export started: 2022_tile_9.tif (Check Google Drive)

📡 Waiting for tasks to complete...
🔄 Current Task Status: ['COMPLETED', 'COMPLETED', 'RUNNING', 'RUNNING', 'READY', 'READY', 'READY', 'READY', 'READY', 'READY']
🔄 Current Task Status: ['COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'RUNNING', 'RUNNING', 'READY']
🔄 Current Task Status: ['COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', 'COMPLETED', '

In [ ]:
import ee
import os
import time

# ✅ Initialize Earth Engine
ee.Initialize(project='ee-felixlundfrandsen')

# ✅ Define the region of interest (Example: California)
roi = ee.Geometry.Rectangle([-120.0, 37.0, -119.5, 37.5])  # Example: California

# ✅ Define available timestamps (from NAIP retrieval)
timestamps = [
    "2004-07-05",
    "2006-07-07",
    "2009-06-22",
    "2012-06-28",
    "2016-06-30",
    "2020-06-26",
    "2022-08-10"  # Latest available
]

# ✅ Set Scale for NAIP (0.6m per pixel)
scale = 0.6  # Best available free resolution

# ✅ Define Google Drive Folder
folder_name = "EarthEngine_NAIP_Timestamped"

# ✅ Function to get the NAIP image for a given date range
def get_naip_image(date):
    start_date = ee.Date(date)
    end_date = start_date.advance(1, 'day')
    
    collection = ee.ImageCollection("USDA/NAIP/DOQQ") \
        .filterDate(start_date, end_date) \
        .filterBounds(roi)
    
    count = collection.size().getInfo()
    if count == 0:
        return None  # No images found for this date range

    return collection.first()

# ✅ Function to monitor a single export task
def wait_for_task(task):
    """Waits for a single task to complete before starting the next one."""
    while task.status()['state'] in ['READY', 'RUNNING']:
        print(f"⏳ Task {task.status()['description']} is {task.status()['state']}...")
        time.sleep(30)  # Wait before checking again

    print(f"✅ Task {task.status()['description']} completed with status: {task.status()['state']}")

# ✅ Export one image per timestamp (ONE AT A TIME)
for date in timestamps:
    year = date[:4]  # Extract the year
    file_name = f"{year}_{date.replace('-', '')}_tile.tif"  # Timestamped filename

    # ✅ Get the NAIP image for the selected date
    naip = get_naip_image(date)

    if naip is None:
        print(f"❌ No available image for {date}, skipping...")
        continue  # Skip to the next date

    naip = naip.select(["R", "G", "B"])

    # ✅ Export to Google Drive (ONE AT A TIME)
    task = ee.batch.Export.image.toDrive(
        image=naip,
        description=file_name,
        folder=folder_name,
        fileNamePrefix=file_name.replace(".tif", ""),
        region=roi,
        crs="EPSG:4326",
        scale=scale,  # ✅ 0.6m per pixel resolution
        maxPixels=1e13  # Avoid pixel limits
    )
    
    print(f"🚀 Starting export: {file_name} (Check Google Drive)")
    task.start()
    
    # ✅ Wait for this task to finish before starting the next
    wait_for_task(task)

print("\n✅ All timestamped exports completed. Check Google Drive in the 'EarthEngine_NAIP_Timestamped' folder.")


🚀 Starting export: 2004_20040705_tile.tif (Check Google Drive)
⏳ Task 2004_20040705_tile.tif is READY...
⏳ Task 2004_20040705_tile.tif is RUNNING...
⏳ Task 2004_20040705_tile.tif is RUNNING...
⏳ Task 2004_20040705_tile.tif is RUNNING...
⏳ Task 2004_20040705_tile.tif is RUNNING...
⏳ Task 2004_20040705_tile.tif is RUNNING...
⏳ Task 2004_20040705_tile.tif is RUNNING...
⏳ Task 2004_20040705_tile.tif is RUNNING...


In [11]:
import ee

# ✅ Initialize Earth Engine
ee.Initialize(project='ee-felixlundfrandsen')

# ✅ Define the region of interest (Example: California)
roi = ee.Geometry.Point([-120.0, 37.0])  # Use a single point or a small area

# ✅ Choose the dataset (Change if needed)
dataset = "USDA/NAIP/DOQQ"  # ✅ NAIP (0.6m aerial imagery)

# ✅ Retrieve all available dates
image_collection = ee.ImageCollection(dataset) \
    .filterBounds(roi) \
    .sort("system:time_start")  # Sort by earliest date

# ✅ Extract dates from the collection
dates = image_collection.aggregate_array("system:time_start").getInfo()

# ✅ Convert timestamps to readable format
import datetime

if dates:
    formatted_dates = [datetime.datetime.utcfromtimestamp(d / 1000).strftime("%Y-%m-%d") for d in dates]
    print("📅 Available Image Dates:")
    for date in formatted_dates:
        print(f"✅ {date}")
else:
    print("❌ No images found for this location.")


📅 Available Image Dates:
✅ 2004-07-03
✅ 2004-07-03
✅ 2004-07-05
✅ 2004-07-05
✅ 2005-06-11
✅ 2005-06-11
✅ 2005-06-11
✅ 2005-06-11
✅ 2006-06-23
✅ 2006-06-23
✅ 2006-07-07
✅ 2006-07-07
✅ 2009-06-22
✅ 2009-06-22
✅ 2009-06-22
✅ 2009-06-22
✅ 2010-06-25
✅ 2010-06-25
✅ 2010-06-25
✅ 2010-06-25
✅ 2012-06-22
✅ 2012-06-22
✅ 2012-06-28
✅ 2012-06-28
✅ 2014-06-22
✅ 2014-06-22
✅ 2014-06-27
✅ 2014-06-27
✅ 2016-06-21
✅ 2016-06-21
✅ 2016-06-30
✅ 2016-06-30
✅ 2018-08-15
✅ 2018-08-15
✅ 2018-09-08
✅ 2018-09-08
✅ 2020-06-16
✅ 2020-06-16
✅ 2020-06-26
✅ 2020-06-26
✅ 2022-06-15
✅ 2022-06-15
✅ 2022-06-20
✅ 2022-08-10


C:\Users\felix\AppData\Local\Temp\ipykernel_24616\2588491142.py:24: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  formatted_dates = [datetime.datetime.utcfromtimestamp(d / 1000).strftime("%Y-%m-%d") for d in dates]


In [1]:
import ee
ee.Authenticate()


True